<a href="https://colab.research.google.com/github/SethJorissen/ANM_anomalies_project/blob/main/ANM_anomalies_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANM anomaly detection project

Tsinghua 2023, produced by Aris Dressino and Seth Jorissen

## 1. initial set up

- fetch data for analysis
- import needed libraries

In [18]:
# fetch phase 1 data from github to colab working directory - run once
# consider using another method if working from local client
%%capture
#import shutil
#shutil.rmtree('/content/phase1')
!svn checkout https://github.com/SethJorissen/ANM_anomalies_project/trunk/phase1

In [ ]:
# needed libraries
import pandas as pd

##2. Exploratory data analysis

- import train and test deta for evaluation
- provide summary statistics and relevant analysis of the data



##3. Anomaly Detection Model

- train the selected model
- provide metrics design

##4. evaluation, prediction and metrics

- evaluate test set predictions
- define KPIs and relevant metrics
- create info-dashboard explaining the findings